# Вебинар 4. Домашнее задание

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [7]:
data = pd.read_csv('02_Data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                     'product_id': 'item_id'},
            inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [8]:
item_features = pd.read_csv('02_Data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [9]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [ ]:
# n_items_before = data_train['item_id'].nunique()

# data_train = prefilter_items(data_train, item_features, take_n_popular=5000)

# n_items_after = data_train['item_id'].nunique()
# print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

In [32]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(2)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [13]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [14]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 1min 8s, sys: 54.6 s, total: 2min 2s
Wall time: 10.9 s


In [82]:
user_id = 443 # np.random.choice(data['user_id'], size=1)[0]
user_id

443

In [83]:
user_item_matrix.loc[user_id, :]

item_id
25671       0.0
26081       0.0
26093       0.0
26190       0.0
26355       0.0
           ... 
17381856    0.0
17382205    0.0
17383227    0.0
17827644    0.0
17829232    0.0
Name: 443, Length: 86865, dtype: float64

In [87]:
df = user_item_matrix.loc[user_id, :].reset_index().groupby('item_id').sum().sort_values(by=user_id, ascending=False).head()
df

,443
item_id,
8011397,30.0
982393,17.0
981760,16.0
9221763,16.0
1057260,14.0


In [89]:
df.reset_index()['item_id'].tolist()

[8011397, 982393, 981760, 9221763, 1057260]

In [72]:
temp = user_item_matrix.loc[user_id, :].reset_index().groupby('item_id').sum().reset_index()
temp[temp[user_id] == temp[user_id].max()]['item_id'].values[0]

8011397

In [74]:
idx = itemid_to_id[8011397]
idx

59737

In [126]:
closest_items = [id_to_itemid[row_id] for row_id, score in model.similar_items(59737, N=5)]
closest_items

[8011397, 8011282, 8011285, 7466806, 13911257]

In [80]:
model.similar_items(59737, N=5)

[(59737, 1.0),
 (59732, 0.9432394),
 (59733, 0.93221045),
 (59644, 0.9053404),
 (81895, 0.87219024)]

In [97]:
def get_top_n_users_items(user_id, N=5):
    df = user_item_matrix.loc[user_id, :].reset_index().groupby('item_id').sum().sort_values(by=user_id, ascending=False).head(N)
    return df.reset_index()['item_id'].tolist()

In [112]:
items_id = get_top_n_users_items(443, N=5)
items_id

[8011397, 982393, 981760, 9221763, 1057260]

In [120]:
for item in items_id:
    print(model.similar_items(itemid_to_id[item], N=2))

[(59737, 1.0), (59732, 0.9432394)]
[(23663, 1.0), (28525, 0.8708966)]
[(23590, 1.0000001), (21476, 0.9441133)]
[(62982, 0.9999998), (56212, 0.86026776)]
[(31960, 1.0000001), (69699, 0.8668676)]


In [123]:
id_to_itemid[31960]

1057260

In [124]:
def get_similar_items_recommendation(user_id:int, N:int=5) -> list:
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    # your_code
    items_id = get_top_n_users_items(user_id, N=N)
    rec = []
    for item in items_id:
        rec.append(model.similar_items(itemid_to_id[item], N=2)[1][0])
#     rec = [id_to_itemid[model.similar_items(itemid_to_id[item], N=2)[1][0]] for item in items_id]

    assert len(rec) == N, 'Количество рекомендаций != {}'.format(N)
    return rec

In [125]:
get_similar_items_recommendation(443, N=5)

[8011282, 1026448, 961792, 6602664, 10200444]

In [118]:
get_similar_items_recommendation(443, N=5)

[59732, 28525, 21476, 56212, 69699]

-----

# Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [18]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# from src.utils import prefilter_items
# from src.metrics import precision_at_k

class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame !!! возможно ошибка - нужен просто датасет взаимодействий (первоначальный - data)
        Матрица взаимодействий user-item
        
    weighting: bool, default True
        Флаг для выполнения взвешивания данных (BM25)
    """
    
    def __init__(self, data:pd.DataFrame, weighting:bool=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать

        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        
        self.id_to_itemid, self.id_to_userid,
        self.itemid_to_id, self.userid_to_id = prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 

        self.model = self.fit(self.user_item_matrix)
        
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def prepare_matrix(data:pd.DataFrame) -> pd.DataFrame:
        # your_code
        user_item_matrix = pd.pivot_table(data, index='user_id', columns='item_id',
                                          values='quantity', # Можно пробовать другие варианты
                                          aggfunc='count', 
                                          fill_value=0
                                         )

        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix:pd.DataFrame) -> (dict, dict, dict, dict):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod    
    def get_top_n_users_items(user_id:int, N:int=5) -> list:
        df = self.user_item_matrix.loc[user_id, :].reset_index().groupby('item_id').sum().sort_values(by=user_id, ascending=False).head(N)
        return df.reset_index()['item_id'].tolist() 
    
    @staticmethod
    def fit(user_item_matrix:pd.DataFrame, n_factors:int=20, regularization:float=0.001, iterations:int=15, num_threads:int=4) -> 'ALS_Model':
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=factors, 
                                        regularization=regularization,
                                        iterations=iterations,  
                                        num_threads=num_threads)
        
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr())
        
        return model

    @staticmethod
    def fit_own_recommender(user_item_matrix:pd.DataFrame) -> 'ItemItem_Model':
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    def get_similar_items_recommendation(self, user_id:int, N:int=5) -> list:
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        # your_code
        items_id = self.get_top_n_users_items(user_id, N=N)
        rec = []
        for item in items_id:
            rec.append(self.model.similar_items(itemid_to_id[item], N=2)[1][0])
#         rec = [id_to_itemid[model.similar_items(itemid_to_id[item], N=2)[1][0]] for item in items_id]

        assert len(rec) == N, 'Количество рекомендаций != {}'.format(N)
        return rec
    
    def get_similar_users_recommendation(self, user_id:int, N:int=5) -> list:
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        # your_code

        assert len(rec) == N, 'Количество рекомендаций != {}'.format(N)
        return rec

----

Проверка, что все работает

In [21]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender